* Predicate-head combo
* Path (from constituent to root)
* Distance( between constituent and predicate) & Predicate specified(TBD)

In [1]:
import os
os.chdir(r'C:\Users\anaverageone\htlt_env\Advanced_NLP')
print(os.getcwd())
import pandas as pd
import numpy as np

C:\Users\anaverageone\htlt_env\Advanced_NLP


In [ ]:
# train_path = 'data/train_split.tsv'
# header_split_df = ['Copy_ID','Sent_ID','ID','FORM','LEMMA','UPOS','XPOS','FEATS','HEAD','DEPREL','DEPS','MISC','UP:PRED','UP:ARGHEADS']
train_path, test_path = 'data/train_split.tsv', 'data/test_split.tsv'
train_df, test_df = pd.read_csv(train_path, sep='\t', encoding='utf-8',quotechar='№'), pd.read_csv(test_path, sep='\t', encoding='utf-8',quotechar='№')


C:\Users\anaverageone\AppData\Local\Temp\ipykernel_13956\123592276.py:4: ParserWarning: Falling back to the 'python' engine because ord(quotechar) > 127, meaning the quotechar is larger than one byte, and the 'c' engine does not support such quotechars; you can avoid this warning by specifying engine='python'.
  train_df, test_df = pd.read_csv(train_path, sep='\t', encoding='utf-8',quotechar='№'), pd.read_csv(test_path, sep='\t', encoding='utf-8',quotechar='№')
C:\Users\anaverageone\AppData\Local\Temp\ipykernel_13956\123592276.py:4: ParserWarning: Falling back to the 'python' engine because ord(quotechar) > 127, meaning the quotechar is larger than one byte, and the 'c' engine does not support such quotechars; you can avoid this warning by specifying engine='python'.
  train_df, test_df = pd.read_csv(train_path, sep='\t', encoding='utf-8',quotechar='№'), pd.read_csv(test_path, sep='\t', encoding='utf-8',quotechar='№')


In [ ]:
train_df, test_df = train_df.astype({'Sent_ID':'int'}), test_df.astype({'Sent_ID':'int'})
train_df, test_df = train_df.astype({'Copy_ID':'int'}), test_df.astype({'Copy_ID':'int'})
train_df, test_df = train_df.astype({'ID':'int'}), test_df.astype({'ID':'int'})

In [14]:
train = train_df[:50]
test = test_df[:50]

In [22]:
feature_dict_list = []
copy_id_list = train['Copy_ID'].unique()

for n in copy_id_list:
    df_copy = train.loc[train['Copy_ID'] == n] # subset_df of train_df for each copy_id
    max_wds_count = df_copy['ID'].max()
        
    for i in range(max_wds_count):
        feature_dict = {}
        paths = []
        if i in df_copy.index:
            head_ix = df_copy.at[i, 'HEAD']
            dep_rel = df_copy.at[i, 'DEPREL']
        else:
                # handle the case where the row with index i does not exist
            head_ix, dep_rel = '0' , '0'

        if int(head_ix) == 0:
            head_form = '0'
        else:
            head_form = df_copy.loc[(df_copy['ID'] == head_ix),'FORM'].squeeze()
                
            # predicate of current phrase
            up_pred = df_copy.loc[(df_copy['UP:PRED'] != '_'),'FORM'].squeeze()

            # Feature - predicate & head combination
            feature_dict['PRED_HEAD']= up_pred + '_' + head_form
            
            # Feature - Path from constituent to root
            while int(head_ix) != 0:
                # head_ix = df_copy.loc[df_copy['ID'] == ,'HEAD'].squeeze()
                head_ix = df_copy.loc[df_copy['ID'] == int(head_ix), 'HEAD']
                if int(head_ix) == 0:
                    dep_rel = 'root'
                else:
                    dep_rel = df_copy.loc[df_copy['ID'] == int(head_ix),'DEPREL'].squeeze()
                paths.append(dep_rel)
                paths = list(dict.fromkeys(paths))
                
            feature_dict['PATH']= paths
            feature_dict_list.append(feature_dict)

In [23]:
print(feature_dict_list)

[{'PRED_HEAD': 'killed_Al', 'PATH': ['root']}, {'PRED_HEAD': 'killed_Al', 'PATH': ['root']}, {'PRED_HEAD': 'killed_Al', 'PATH': ['root']}, {'PRED_HEAD': 'killed_forces', 'PATH': ['parataxis', 'root']}, {'PRED_HEAD': 'killed_killed', 'PATH': ['root']}, {'PRED_HEAD': 'killed_Al', 'PATH': ['root']}, {'PRED_HEAD': 'killed_killed', 'PATH': ['root']}, {'PRED_HEAD': 'killed_Shaikh', 'PATH': ['parataxis', 'root']}, {'PRED_HEAD': 'killed_Shaikh', 'PATH': ['parataxis', 'root']}, {'PRED_HEAD': 'killed_Shaikh', 'PATH': ['parataxis', 'root']}, {'PRED_HEAD': 'killed_Shaikh', 'PATH': ['parataxis', 'root']}, {'PRED_HEAD': 'killed_Shaikh', 'PATH': ['parataxis', 'root']}, {'PRED_HEAD': 'killed_preacher', 'PATH': ['obj', 'parataxis', 'root']}, {'PRED_HEAD': 'killed_Shaikh', 'PATH': ['parataxis', 'root']}, {'PRED_HEAD': 'killed_mosque', 'PATH': ['parataxis', 'root']}, {'PRED_HEAD': 'killed_mosque', 'PATH': ['parataxis', 'root']}, {'PRED_HEAD': 'killed_killed', 'PATH': ['root']}, {'PRED_HEAD': 'killed_town

In [21]:
mylist = ["a", "b", "a", "c", "c"]
mylist = list(dict.fromkeys(mylist))
print(type(mylist))

<class 'list'>


In [ ]:
# feature_dict_list = []
# copy_id_list = test['Copy_ID'].unique()
# for n in copy_id_list:
#         df_copy = test.loc[test['Copy_ID'] == n]  # subset_df of train_df for each copy_id
#         max_wds_count = df_copy['ID'].max()
# #         df_copy['PATH'] = '0'
        
#         for i in range(max_wds_count):
#             feature_dict = {}
#             paths = []
#             head_ix = df_copy.at[i, 'HEAD']
#             dep_rel = df_copy.at[i, 'DEPREL'] 
#             paths.append(dep_rel)
#             # dep_rel = ":".join(list(filter(lambda x: x.isalpha(), df_copy.iloc[i]['DEPS'].split(":")))
                
#             while int(head_ix) != 0:
#                 head_ix = df_copy.loc[df_copy['ID'] == head_ix,'HEAD'].squeeze()
#                 if int(head_ix) == 0:
#                     dep_rel = 'root'
#                 else:
#                     dep_rel = df_copy.loc[df_copy['ID'] == head_ix,'DEPREL'].squeeze()
#                 paths.append(dep_rel)
#                 paths = pd.Series(paths).drop_duplicates().tolist()
            
#             print(paths)